In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 970.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 11.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: p

In [4]:
from datasets import load_dataset
def load_qa_dataset(dataset_name = 'kikikara/ko_QA_dataset'):
    dataset = load_dataset(dataset_name)
    documents = []
    for item in dataset['train']:
        doc = {
            'question':item['input'],
            'answer':item['output'],
            'metadata':{
                'source':'huggingface',
                'dataset':dataset_name,
                'split':'train'
            }
        }
        documents.append(doc)
    return documents

In [8]:
qa_docs = load_qa_dataset()

Found cached dataset parquet (/root/.cache/huggingface/datasets/kikikara___parquet/kikikara--ko_QA_dataset-68c138cc5c8eb15f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
from typing import List, Dict,Any
def create_document_chunks(documents: List[Dict[str,Any]],
                           chunk_size: int = 1000,
                           overlap:int=200) -> List[Dict[str,Any]]:
    chunked_docs=[]
    for doc in documents:
        content = f"질문: {doc['question']}\n답변: {doc['answer']}"
            
        # 내용을 청크로 분할
        start = 0
        while start < len(content):
            end = start + chunk_size
            chunk = content[start:end]
            
            # 청크 정보 저장
            chunk_doc = {
                "content": chunk,
                "metadata": {
                    **doc.get("metadata", {}),
                    "chunk_index": len(chunked_docs),
                    "start_pos": start,
                    "end_pos": end
                }
            }
            chunked_docs.append(chunk_doc)
            
            # 다음 청크의 시작 위치 계산 (중복 고려)
            start = end - overlap
        
    return chunked_docs 

In [14]:
chunked_docs = create_document_chunks(qa_docs)

In [7]:
from sentence_transformers import SentenceTransformer
from pathlib import Path
from typing import List, Dict, Any
def koreanEmbeddingModel(embedding_model_name = 'jhgan/ko-sroberta-multitask',index_path = './faiss_index'):
    model =  SentenceTransformer(embedding_model_name)
    dimension = model.get_sentence_embedding_dimension()
    index_path = Path(index_path)
    index_path.mkdir(parents=True, exist_ok=True)
    # FAISS 인덱스 초기화
    index = faiss.IndexFlatL2(dimension)
    documents = []
    metadata = []     

In [15]:
documents = [doc['content'] for doc in chunked_docs]
metadata = [doc['metadata'] for doc in chunked_docs]